In [1]:
import torch
import numpy as np
from transformers import AutoTokenizer, Blip2Model
from PIL import Image
import time
import json


C:\Users\msubr\Anaconda3\envs\bert\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
processor = AutoTokenizer.from_pretrained("Salesforce/blip2-opt-2.7b")
# by default `from_pretrained` loads the weights in float32
# we load in float16 instead to save memory
model = Blip2Model.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16)

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████| 2/2 [00:08<00:00,  4.17s/it]


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print(device)

cuda


In [4]:
def load_images(files, bs=16):
    init = 0
    while(init < len(files)):
        yield files[init:init + bs]
        init += bs
        
def save_features(files, features):
    path = f"d:/data/UCF-crime/Anomaly-videos-description-features/{label}/"
    np.save(path + f, features.cpu().detach().numpy())
        

In [9]:
label = "Vandalism"
with open(f"desc-{label}.txt", 'r') as f:
    descriptions = json.load(f)


In [10]:
for f, desc in descriptions:
    inputs = processor([desc[0][0:-1]], padding=True, return_tensors="pt").to(device)    
    features = text_features = model.get_text_features(**inputs, return_dict=True)
    T = torch.permute(features['past_key_values'][-1][0], (0, 2, 1, 3))
    T = torch.reshape(T, (1, -1, 32*80))
    save_features([f], T)  